In [24]:
import pandas as pd
import numpy as np
import os
import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import *

In [2]:
seed = 7

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Collect data

In [4]:
root_data='comodity-price-prediction-penyisihan-arkavidia-9'
hrg_bhn_pangan='Harga Bahan Pangan'
folder_train='train'
folder_mata_uang='Mata Uang',
folder_google_trend='Google Trend'

In [28]:
tanggal_awal=dt.datetime(year=2022,month=1,day=1)
tanggal_akhir=dt.datetime(year=2022,month=1,day=3)

In [6]:
list_bhn_pgn=[
    'Bawang Merah',         # bawang merah
    'Bawang Putih Bonggol', # bawang putih
    'Beras Medium',         # beras
    'Beras Premium',        # beras
    'Cabai Merah Keriting', # cabai merah
    'Cabai Rawit Merah',    # cabai rawit
    'Daging Ayam Ras',      # daging ayam
    'Daging Sapi Murni',    # daging sapi
    'Gula Konsumsi',        # gula
    'Minyak Goreng Curah',  # minyak goreng
    'Minyak Goreng Kemasan Sederhana',  #minyak goreng
    'Telur Ayam Ras',       # telur ayam
    'Tepung Terigu (Curah)',# tepung terigu
]

In [35]:
list_google_trend={
    'Bawang Merah': 'bawang merah',
    'Bawang Putih Bonggol': 'bawang putih',
    'Beras Medium':'beras',
    'Beras Premium':'beras',
    'Cabai Merah Keriting':'cabai merah',
    'Cabai Rawit Merah':'cabai rawit',
    'Daging Ayam Ras':'daging ayam',
    'Daging Sapi Murni':'daging sapi',
    'Gula Konsumsi':'gula',
    'Minyak Goreng Curah':'minyak goreng',
    'Minyak Goreng Kemasan Sederhana':'minyak goreng',
    'Telur Ayam Ras':'telur ayam',
    'Tepung Terigu (Curah)':'tepung terigu',
}

In [7]:
list_nama_daerah=[
    'Aceh',
    'Bali',
    'Banten',
    'Bengkulu',
    'DI Yogyakarta',
    'DKI Jakarta',
    'Gorontalo',
    'Jambi',
    'Jawa Barat',
    'Jawa Tengah',
    'Jawa Timur',
    'Kalimantan Barat',
    'Kalimantan Selatan',
    'Kalimantan Tengah',
    'Kalimantan Timur',
    'Kalimantan Utara',
    'Kepulauan Bangka Belitung',
    'Kepulauan Riau',
    'Lampung',
    'Maluku Utara',
    'Maluku',
    'Nusa Tenggara Barat',
    'Nusa Tenggara Timur',
    'Papua Barat',
    'Papua',
    'Riau',
    'Sulawesi Barat',
    'Sulawesi Selatan',
    'Sulawesi Tengah',
    'Sulawesi Tenggara',
    'Sulawesi Utara',
    'Sumatera Barat',
    'Sumatera Selatan',
    'Sumatera Utara'
]

In [8]:
path_data_source=os.path.join(root_data,hrg_bhn_pangan,folder_train)

In [36]:
data_bhn_pgn={
    'Tanggal':[],
    'Bahan_pangan':[],
    'Daerah':[],
    'Harga_lag_1':[],
    'Harga_lag_7':[],
    'Harga_lag_30':[],
    'Google_trends':[],
    'MYRUSD':[],
    'SGDUSD':[],
    'THBUSD':[],
    'USDIDR':[],
    'Harga':[]
}
df_bhn_pgn_all=pd.DataFrame(data_bhn_pgn)

In [46]:
for date in [tanggal_awal + dt.timedelta(days=x) for x in range((tanggal_akhir - tanggal_awal).days + 1)]:
    for nama_bhn_pgn in list_bhn_pgn:
        file_bhn_pgn=os.path.join(path_data_source, nama_bhn_pgn+'.csv')
        df_bhn_pgn=pd.read_csv(file_bhn_pgn)
        try:
            df_bhn_pgn['Date']=pd.to_datetime(df_bhn_pgn['Date'])
        except Exception as e:
            df_bhn_pgn['Tanggal']=pd.to_datetime(df_bhn_pgn['Tanggal'])
        # df_bhn_pgn=df_bhn_pgn[df_bhn_pgn['Date']==date]
        yesterday = date - timedelta(days=1)
        week_ago = date - timedelta(days=7)
        month_ago = date - relativedelta(months=1)
        for daerah in list_nama_daerah:
            # df_bhn_pgn_daerah=df_bhn_pgn[daerah]
            try:
                gt_path=os.path.join(root_data,folder_google_trend,list_google_trend[nama_bhn_pgn],daerah+'.csv')
                df_gt=pd.read_csv(gt_path)
                df_gt['Date']=pd.to_datetime(df_gt['Date'])
                # print(gt_path)
            except Exception as e:
                df_gt=None
            new_data={
                'Tanggal':date,
                'Bahan_pangan':nama_bhn_pgn,
                'Daerah':daerah,
                'Harga_lag_1':df_bhn_pgn.query('Date == @yesterday')[daerah].values[0] if not df_bhn_pgn.query('Date == @yesterday').empty else 0,
                'Harga_lag_7':df_bhn_pgn.query('Date == @week_ago')[daerah].values[0] if not df_bhn_pgn.query('Date == @week_ago').empty else 0,
                'Harga_lag_30':df_bhn_pgn.query('Date == @month_ago')[daerah].values[0] if not df_bhn_pgn.query('Date == @month_ago').empty else 0,
                'Google_trends':df_gt.query('Date == @date')[list_google_trend[nama_bhn_pgn]].values[0] if df_gt is not None and not df_gt.query('Date == @date').empty else 0,
                'MYRUSD':0,
                'SGDUSD':0,
                'THBUSD':0,
                'USDIDR':0,
                'Harga':df_bhn_pgn.query('Date == @date')[daerah].values[0] if not df_bhn_pgn.query('Date == @date').empty else None
            }
            df_bhn_pgn_all = df_bhn_pgn_all._append(new_data, ignore_index=True)

In [47]:
df_bhn_pgn_all.to_csv('test.csv')

In [56]:
file_bhn_pgn=os.path.join(path_data_source, list_bhn_pgn[0]+'.csv')
df_bhn_pgn=pd.read_csv(file_bhn_pgn)
gt_path=os.path.join(root_data,folder_google_trend,list_google_trend[list_bhn_pgn[0]],list_nama_daerah[0]+'.csv')
df_gt=pd.read_csv(gt_path)
df_gt.query('Date == @date')['bawang merah'].values[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [51]:
list_bhn_pgn[0]

'Bawang Merah'

In [53]:
list_google_trend['Bawang Merah']

'bawang merah'

In [54]:
list_google_trend[list_bhn_pgn[0]]

'bawang merah'